# Create plaquette using the RPNG format

## 1. RPNG format

```
-z1- -z2- -z3- -z4-
rpng rpng rpng rpng
```

(r) data qubit reset basis or -  
(p) data basis for the controlled operation (x means CNOT controlled on the ancilla and targeting the data qubit, y means CZ)  
(n) time step  
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- the ancilla is always initialized in $\ket{+}$ and measured in the X basis
- the ancilla is always the control qubit for the CNOT and CZ gates

## 2. Extended RPNG format

```
z0z5 -xz1- -xz2- -xz3- -xz4-
pnpn rppng rppng rppng rppng
```

(p) ancilla init basis  
(n) time step  
(p) ancilla measure basis  
(n) time step

(r) data qubit reset basis or -  
(pp) ancill-data 2-qubit bases (xz means CNOT targeting the ancilla)  
(n) time step  
(g) data qubit measure basis or h or -

Time step of r same as ancilla reset. Time step of g same as ancilla measurement.

## 3. Write the functionality

In [1]:
from typing import Literal
from pathlib import Path
import stim
import matplotlib.pyplot as plt

from tqec import (
    Plaquette,
    PlaquetteQubits,
    QubitMap,
    ScheduledCircuit,
    SquarePlaquetteQubits,
)

ASSETS_FOLDER = Path("../../assets/").resolve()

ImportError: cannot import name 'QubitMap' from 'tqec' (/data/giangiac/tqec/lib/python3.10/site-packages/tqec/__init__.py)

In [59]:
def validate_rpng_string(rpng_string: str) -> int:
    """Check the validity of a RPNG string
    
    Return values:
    0 -- invalide rpng string
    1 -- valide rpng string in the simplified format 
    2 -- valide rpng string in the extended format 
    """
    q_values = rpng_string.split(' ')
    acceptable_r = ['-', 'x', 'y', 'z', 'h']
    acceptable_p = ['-', 'x', 'y', 'z']
    acceptable_g = acceptable_r
    acceptable_r_ancilla = ['x', 'y', 'z']
    acceptable_g_ancilla = acceptable_r_ancilla
    init_time = 0
    meas_time = 6
    if len(q_values) == 4:
        times = []
        for i, v in enumerate(q_values):
            if len(v) != 4:
                return 0
            # Collect times.
            if v[2] != '-':
                if v[2].isdigit() and int(v[2]) > init_time and int(v[2]) < meas_time:
                    times.append(int(v[2]))
                else:
                    return 0
            # In absence of a time, we impose that RPNG must be '----'.
            elif v != '----':
                return 0
            is_valid = v[0] in acceptable_r and \
                       v[1] in acceptable_p and \
                       v[3] in acceptable_g
            if not is_valid:
                return 0
        # Confirm unique time and that either 0, 2, or 4 data are involved in 2Q ops.
        if len(times) in [0,2,4] and len(times) == len(set(times)):
            return 1
        else:
            return 0
    elif len(q_values) == 5:
        # Value for the ancilla qubit.
        v = q_values[0]
        if len(v) != 4:
            return 0
        # Update init time.
        if v[1].isdigit() and int(v[1]) >= init_time and int(v[1]) < meas_time:
            init_time = int(v[1])
        else:
            return 0
        # Update meas time.
        if v[3].isdigit() and int(v[3]) > init_time and int(v[3]) <= meas_time:
            meas_time = int(v[3])
        else:
            return 0
        is_valid = v[0] in acceptable_r_ancilla and \
                   v[2] in acceptable_g_ancilla
        if not is_valid:
            return 0
        #TBD: print(f'valid value for the ancilla (init={init_time}, meas={meas_time})')
        # values for the data qubits.
        times = []
        for i, v in enumerate(q_values[1:]):
            if len(v) != 5:
                return 0
            # Collect times.
            if v[3] != '-':
                if v[3].isdigit() and int(v[3]) > init_time and int(v[3]) < meas_time:
                    times.append(int(v[3]))
                else:
                    return 0
            # In absence of a time, we impose that RPNG must be '-----'.
            elif v != '-----':
                return 0
            #TBD: print(f'time is correct: {times[-1]}')
            is_valid = v[0] in acceptable_r and \
                       v[1] in acceptable_p and \
                       v[2] in acceptable_p and \
                       v[4] in acceptable_g
            #TBD: print(f'is_valid = {is_valid}')
            if not is_valid:
                return 0
        # Confirm unique time and that either 0, 2, or 4 data are involved in 2Q ops.
        #TBD: print(f'unique times? {times}')
        if len(times) in [0,2,4] and len(times) == len(set(times)):
            return 2
        else:
            return 0
    else:
        return 0


def create_plaquette_from_rpng_string(rpng_string: str, qubits: PlaquetteQubits) -> Plaquette:
    """Create a plaquette from the RPNG format"""
    format = validate_rpng_string(rpng_string)
    scheduled_circuit_as_list = ['TICK\n']*7 # for time=0, 1, ..., 6
    if format == 1:
        print('simplified RPNG format')
    elif format == 2:
        print('extended RPNG format')
    elif format == 0:
        raise ValueError(f'invalide rpng string "{rpng_string}"')
    print(scheduled_circuit_as_list)
    q_map = QubitMap.from_qubits(qubits)
    scheduled_circuit = ScheduledCircuit.from_circuit([stim.Circuit(k) for k in scheduled_circuit_as_list], qubit_map = q_map)
    return None
    #return Plaquette(name = 'test', qubits = qubits, circuit = None)

rpng = '-z5- -x2- -x3- -z1-' # correct
qubits = SquarePlaquetteQubits()
ancilla = qubits.syndrome_qubits
print(ancilla)
print(type(qubits), '\n')

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
except ValueError as err:
    print(f'{err}')


[GridQubit(0, 0)]
<class 'tqec.plaquette.qubit.SquarePlaquetteQubits'> 

simplified RPNG format
['TICK\n', 'TICK\n', 'TICK\n', 'TICK\n', 'TICK\n', 'TICK\n', 'TICK\n']


NameError: name 'QubitMap' is not defined

In [ ]:
# Simplified RPNG format.
rpng = '---- ---- ---- ---- ----' # wrong number of values
rpng = '---- ---- --- ----' # wrong length of values
rpng = '---- ---- ---- ----' # correct but uneventful
rpng = '-z1- -z2- -z3- -z4-' # correct
rpng = '-z1- -z2- ---- -z4-' # wrong number of 2Q gates
rpng = '-z1- -z4- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z1- -z6- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z5- -x2- -x3- -z1-' # correct

# Extended RPNG format.
rpng = 'z0z5 -xz1- -xz2- -xz3- -xz4-' # wrong times for the 2Q gates
rpng = 'z0z3 -xz1- ----- -xz2- -----' # correct
rpng = 'z3z0 -xx1- ----- -xz2- ----' # wrong meas time
rpng = 'z0z5 -xz1- -xz2- -xz3- -xz4-' # correct

qubits = SquarePlaquetteQubits()
print(type(qubits), '\n')

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
except ValueError as err:
    print(f'{err}')
